In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import sys
sys.path.append('/content/drive/MyDrive/CTABGANforClickThrough')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install dython

In [3]:
!pip install --upgrade numpy
!pip install --upgrade scipy sklearn
!pip install joblib

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
from module.model.ctab_gan import CTABGAN
from module.evaluation.evaluation import get_utility_metrics,stat_sim, privacy_metrics
import numpy as np
import pandas as pd
import glob

In [5]:
# Specifying the replication number
num_exp = 1
# Specifying the name of the dataset used
dataset = "minismalltrain"
# Specifying the path of the dataset used
transformed_real_path_root = "/content/drive/MyDrive/CTABGANforClickThrough/dataset/Transformed"
real_path = "/content/drive/MyDrive/CTABGANforClickThrough/dataset/minismalltrain.csv"
# Specifying the root directory for storing generated data
fake_file_root = "/content/drive/MyDrive/CTABGANforClickThrough/dataset/Fake_Datasets"
transformed_fake_file_root = "/content/drive/MyDrive/CTABGANforClickThrough/dataset/Fake_Datasets/Transformed"

categ_col = ['click', 'C1', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19',
             'C20', 'C21', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'banner_pos',
             'app_category', 'device_id', 'device_ip', 'device_model',  'device_type', 'device_conn_type']
new_cat_col = ['click', 'C1', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19',
             'C20', 'C21', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'banner_pos',
             'app_category', 'device_id', 'device_ip', 'device_model',  'device_type', 'device_conn_type', 'Year',
               'Month', 'Day', 'Hour']
integ_col = []
mixed_col = {}
target = {"Classification": 'click'}

# Initializing the synthesizer object and specifying input parameters
# Notice: If you have continuous variable, you do not need to explicitly assign it. It will be treated like
# that by default
ctabgan =  CTABGAN(raw_csv_path = real_path,
                 test_ratio = 0.20,
                 categorical_columns = categ_col,
                 log_columns = [],
                 mixed_columns= mixed_col,
                 integer_columns = integ_col,
                 target = target,
                   processed_csv_path = transformed_real_path_root + "/" +"processed_data.csv",
                 num_epochs = 5,
                   chunk_size=1000)
# Fitting the synthesizer to the training dataset and generating synthetic data
for i in range(num_exp):
    ctabgan.fit()
    syn1, syn2 = ctabgan.generate_samples()
    syn1.to_csv(transformed_fake_file_root + "/" + dataset+"_fake_{exp}.csv".format(exp=i), index= False)
    syn2.to_csv(fake_file_root+"/"+dataset+"/"+ dataset+"_fake_{exp}.csv".format(exp=i), index= False)

Finished reading
We have 81 chunks in total.


  0%|          | 0/1 [00:00<?, ?it/s]

Starting epoch 1
Training 0-th chunk.
Training 1-th chunk.
Training 2-th chunk.
Training 3-th chunk.
Training 4-th chunk.
Training 5-th chunk.
Training 6-th chunk.
Training 7-th chunk.
Training 8-th chunk.
Training 9-th chunk.
Training 10-th chunk.
Training 11-th chunk.
Training 12-th chunk.
Training 13-th chunk.
Training 14-th chunk.
Training 15-th chunk.
Training 16-th chunk.
Training 17-th chunk.
Training 18-th chunk.
Training 19-th chunk.
Training 20-th chunk.
Training 21-th chunk.
Training 22-th chunk.
Training 23-th chunk.
Training 24-th chunk.
Training 25-th chunk.
Training 26-th chunk.
Training 27-th chunk.
Training 28-th chunk.
Training 29-th chunk.
Training 30-th chunk.
Training 31-th chunk.
Training 32-th chunk.
Training 33-th chunk.
Training 34-th chunk.
Training 35-th chunk.
Training 36-th chunk.
Training 37-th chunk.
Training 38-th chunk.
Training 39-th chunk.
Training 40-th chunk.
Training 41-th chunk.
Training 42-th chunk.
Training 43-th chunk.
Training 44-th chunk.
Tra

100%|██████████| 1/1 [00:35<00:00, 35.36s/it]


Finished training in 51.26062083244324 seconds.


In [ ]:
from module.evaluation.evaluation import get_utility_metrics,stat_sim, privacy_metrics
real_afterprep = "/content/drive/MyDrive/CTABGANforClickThrough/module/data_processing/training_data.csv"
fake_paths = glob.glob(transformed_fake_file_root+"/"+"*")

# Specifying the list of classifiers to conduct ML utility evaluation
classifiers_list = ["lr","dt","rf","mlp","svm"]

# Storing and presenting the results as a dataframe
result_mat = get_utility_metrics(real_afterprep,fake_paths,"MinMax", classifiers_list, test_ratio = 0.20)
result_df  = pd.DataFrame(result_mat,columns=["Acc","AUC","F1_Score"])
result_df.index = classifiers_list
result_df

In [6]:
from module.evaluation.evaluation import get_utility_metrics,stat_sim, privacy_metrics
# Specifying the categorical columns of the dataset used
real_afterprep = "/content/drive/MyDrive/CTABGANforClickThrough/module/data_processing/training_data.csv"
fake_paths = glob.glob(transformed_fake_file_root+"/"+"*")
new_cat_col = ['click', 'C1', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19',
             'C20', 'C21', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'banner_pos',
             'app_category', 'device_id', 'device_ip', 'device_model',  'device_type', 'device_conn_type', 'Year',
               'Month', 'Day', 'Hour']
# Storing and presenting the results as a dataframe
stat_res_avg = []
for fake_path in fake_paths:
    stat_res = stat_sim(real_afterprep, fake_path, new_cat_col)
    stat_res_avg.append(stat_res)

stat_columns = ["Average WD (Continuous Columns","Average JSD (Categorical Columns)","Correlation Distance"]
stat_results = pd.DataFrame(np.array(stat_res_avg).mean(axis=0).reshape(1,3),columns=stat_columns)
stat_results

,Average WD (Continuous Columns,Average JSD (Categorical Columns),Correlation Distance
0,NaN,0.322017,6.647104


In [7]:
# Storing and presenting the results as a dataframe
priv_res_avg = []
for fake_path in fake_paths:
    priv_res = privacy_metrics(real_afterprep, fake_path)
    priv_res_avg.append(priv_res)

privacy_columns = ["DCR between Real and Fake (5th perc)","DCR within Real(5th perc)","DCR within Fake (5th perc)","NNDR between Real and Fake (5th perc)","NNDR within Real (5th perc)","NNDR within Fake (5th perc)"]
privacy_results = pd.DataFrame(np.array(priv_res_avg).mean(axis=0).reshape(1,6),columns=privacy_columns)
privacy_results

,DCR between Real and Fake (5th perc),DCR within Real(5th perc),DCR within Fake (5th perc),NNDR between Real and Fake (5th perc),NNDR within Real (5th perc),NNDR within Fake (5th perc)
0,2.531045,0.18392,2.556155,0.838522,0.25,0.803605
